In [1]:
%matplotlib qt

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
# process prediction and visualize

import numpy as np
#import matplotlib.pyplot as plt


pred = np.load('/Users/carotenuto/Documents/GitHub/3d-facial-landmarks-omfs/MeshCNN/datasets/headspace_cust1/predictions.pkl')
pred = np.reshape(pred, (68,3))
print(pred.shape)

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt



fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = pred[:,0]
y = pred[:,1]
z = pred[:,2]



ax.scatter(x, y, z, c='r', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

ValueError: Cannot load file containing pickled data when allow_pickle=False

In [1]:
import open3d as o3d

def visualize(mesh):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(mesh)
    vis.run()
    vis.destroy_window()

def main():
    mesh = o3d.io.read_triangle_mesh("/Users/carotenuto/Documents/GitHub/3d-facial-landmarks-omfs/scripts/130926121017_reduced.obj")
    visualize(mesh)

main()

ModuleNotFoundError: No module named 'open3d'

In [8]:
import plotly
from mpl_toolkits.mplot3d import Axes3D


In [9]:

import numpy as np
import meshio  
import plotly.graph_objects as go

In [24]:
#NotebookApp.iopub_data_rate_limit=100000000.0

NameError: name 'NotebookApp' is not defined

In [56]:
with open('/Users/carotenuto/Documents/GitHub/3d-facial-landmarks-omfs/MeshCNN/datasets/headspace_cust1/00009/test/130926122455.obj','r') as file:
#with open('/Users/carotenuto/Master Radboud/MscProj/headspace-orig/00009/130926122455.obj','r') as file:
    obj_data = file.read()
#print(obj_data)

In [57]:

def obj_data_to_mesh3d(odata):
    # odata is the string read from an obj file
    vertices = []
    faces = []
    lines = odata.splitlines()   
   
    for line in lines:
        slist = line.split()
        if slist:
            if slist[0] == 'v':
                vertex = np.array(slist[1:], dtype=float)
                vertices.append(vertex)
            elif slist[0] == 'f':
                face = []
                for k in range(1, len(slist)):
                    face.append([int(s) for s in slist[k].replace('//','/').split('/')])
                if len(face) > 3: # triangulate the n-polyonal face, n>3
                    faces.extend([[face[0][0]-1, face[k][0]-1, face[k+1][0]-1] for k in range(1, len(face)-1)])
                else:    
                    faces.append([face[j][0]-1 for j in range(len(face))])
            else: pass
    
    
    return np.array(vertices), np.array(faces) 

In [58]:
vertices, faces = obj_data_to_mesh3d(obj_data)


In [59]:

from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)

In [60]:

vertices.shape

(3994, 3)

In [61]:

x, y, z = vertices[:,:3].T
I, J, K = faces.T

mesh = go.Mesh3d(
            x=x,
            y=y,
            z=z,
            vertexcolor=vertices[:, 3:], #the color codes must be triplets of floats  in [0,1]!!                      
            i=I,
            j=J,
            k=K,
            name='',
            showscale=False)

In [62]:

layout = go.Layout(title='Mesh3d from a Wavefront obj file',
                   font=dict(size=14, color='black'),
                   #width=900,
                   #height=800,
                   #scene=dict(xaxis=dict(visible=False),
                   #           yaxis=dict(visible=False),  
                   #           zaxis=dict(visible=False), 
                   #           aspectratio=dict(x=1.5,
                   #                            y=0.9,
                   #                            z=0.5
                   #                      ),
                   #           camera=dict(eye=dict(x=1., y=1., z=0.5)),
                   #     ), 
                  #paper_bgcolor='rgb(235,235,235)',
                  #margin=dict(t=175)
                  ) 

In [63]:

pl_mygray= [[0.0, 'rgb(112, 123, 143)'],
           [0.25, 'rgb(133, 153, 165)'],
           [0.5,  'rgb(156, 184, 188)'],
           [0.75, 'rgb(185, 210, 210)'],
           [1.0,  'rgb(220, 233, 233)'],
           ]

In [64]:
mesh.update(intensity=z, colorscale=pl_mygray)
mesh.pop('vertexcolor')
layout.update(paper_bgcolor='rgb(50,50,50)', 
              font=dict(size=14, color='white'))
newfig = go.Figure(data=[mesh], layout=layout)
#iplot(newfig)

In [65]:
import pickle
#import torch

preds00009 = '/Users/carotenuto/Documents/GitHub/3d-facial-landmarks-omfs/MeshCNN/datasets/headspace_cust1/predictions.pkl'
true = '/Users/carotenuto/Documents/GitHub/3d-facial-landmarks-omfs/MeshCNN/datasets/headspace_cust1/ldmks.pkl'
isGroundTruth = True
foldernum = '0009'


    
print(coords.shape)
if isGroundTruth:
    with open(true, 'rb') as file:
        coords = pickle.load(file)
    
    for i in range(coords.shape[0]):
        if int(coords[i, 0, 0]) == int(foldernum):
            ldmks_idx = i
            break

    coords = coords[ldmks_idx, 1:, :]
else:
    with open(preds00009, 'rb') as file:
        coords = pickle.load(file)
    coords = np.reshape(coords, (68,3))
    

print(coords.shape)

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt



x = coords[:,0]
y = coords[:,1]
z = coords[:,2]


fig = go.Figure(data=[mesh], layout=layout)
fig.add_scatter3d(
    x=x,
    y=y,
z=z)

iplot(fig)

(68, 3)
(68, 3)
